In [105]:
import subprocess
import threading
import sys
import time
import signal

In [106]:
def run_preprocessor(exe_path):
    try:
        process = subprocess.Popen(
            exe_path,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            stdin=subprocess.PIPE,
            universal_newlines=True
        )

        while process.poll() is None:
            process.stdin.write('.\n')
            process.stdin.flush()

        process.wait()

    except:
        pass

In [107]:
def compile_fortran():
    compile_command = [
        "gfortran",
        "-o", "run_plasRxn_v2.exe",
        "dvode_f90_m.F90",
        "zdplaskin_m.F90",
        "run_plasRxn_v2.F90",
        "bolsig_x86_64_g.dll"
    ]

    try:
        # 컴파일 명령어 실행
        result = subprocess.run(compile_command, check=True, capture_output=True, text=True)
    except subprocess.CalledProcessError as e:
        # 컴파일 중 오류가 발생하면 오류 메시지 출력
        print("컴파일 실패!")
        print("오류 메시지:")
        print(e.stderr)
    except FileNotFoundError:
        # gfortran이 설치되어 있지 않거나 PATH에 없는 경우
        print("gfortran을 찾을 수 없습니다. gfortran이 설치되어 있고 시스템 PATH에 추가되어 있는지 확인하세요.")

In [108]:
def run_executable(exe_path):
    try:
        # 실행 파일 실행
        process = subprocess.Popen(
            exe_path,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
            bufsize=1
        )

        # 실시간으로 출력 읽기
        while True:
            output = process.stdout.readline()
            if "ZDPlasKin INFO: the density of species not configured for BOLSIG+ solver exceeds 1.00D+00" in output:
                print("지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.")
                process.send_signal(signal.SIGTERM)
                break
            if output:
                print(output.strip())

        # 프로세스 종료 대기
        return_code = process.wait()

    except:
        pass

In [109]:
exe_path = r'C:\Users\USER\repos_python\Project\DBD_Plasma\ZDplaskin\CH4_DBD_Plasma\ZDPlaskin\preprocessor.exe'
run_preprocessor(exe_path)




In [110]:
compile_fortran()

In [111]:
exe_path="run_plasRxn_v2.exe"
run_executable(exe_path)


CH4 DBD Plasma Reactor

ZDPlasKin (version 2.0a) INIT:
species        ... 52
reactions      ... 491
BOLSIG+ loader ... bolsigdb.dat : 26 species & 249 collisions
species  link  ... 26
process  link  ... 239
ZDPlasKin INFO: reset data and configuration
ZDPlasKin INFO: set accuracy 1.00D-10 (absolute) & 1.00D-05 (relative)
ZDPlasKin INIT DONE

ZDPlasKin INFO: set autosave in QTplaskin format ON ...
Time_s           E         H^+         CH3
CH        C2H4   C2H6(V13)       C4H9H
C2H4(V1)       CH2^+       CH3^+    CH4(V13)
C3H4^+        C3H7        C2H3      C3H8^+
C2H2        C3H8    C3H8(V1)     C3H6(V)
C2H2(V13)        C3H4      C2H2^+      C3H7^+
C3H6^+      C2H6^+         CH2      C2H5^+
C4H9          H2        H3^+        C2H5
C2H6(V24)    C2H4(V2)    C2H2(V2)       C2H^+
C3H5^+      C2H3^+      C2H4^+       CH5^+
H2^+    C3H8(V2)        C3H5         C2H
C3H6         CH4           H    C2H2(V5)
CH4(V24)       C5H12        C2H6        CH^+
CH4^+
0.0000E+00  1.0000E+00  0.0000E+00  